In [19]:
!cd

C:\Users\piranavan\PycharmProjects\skinDr


In [29]:
import sys
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend

import tensorflow as tf
import numpy as np
from keras.preprocessing import image

In [30]:
print()
print(f"Python {sys.version}")
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")


Python 3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)]
GPU is NOT AVAILABLE


In [31]:
classes = ['Melanoma',
           'Nevus']

In [32]:
img_width = 224
img_height = 224

training_dataset = 'data_set/main_train'
testing_dataset = 'data_set/main_test'

epochs = 50
batch_size = 20

if backend.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    

train_datagen = ImageDataGenerator(rescale= 1. /255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. /255)

train_gen = train_datagen.flow_from_directory(training_dataset, classes=classes, target_size=(img_width, img_height), 
                                              batch_size=batch_size, class_mode='binary')

test_gen = test_datagen.flow_from_directory(testing_dataset, classes=classes, target_size=(img_width, img_height), 
                                              batch_size=batch_size, class_mode='binary')


Found 1667 images belonging to 2 classes.
Found 283 images belonging to 2 classes.


In [40]:
# test_datagen.shape

In [33]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_7 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 32)      0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_7 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_8 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 52, 32)       

In [42]:
# model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit_generator(train_gen,
#                    steps_per_epoch= 1000 // batch_size,
#                    epochs=epochs,
#                    validation_data=test_gen,
#                    validation_steps= 50 // batch_size)
model.fit_generator(train_gen,
                   epochs=epochs,
                   validation_data=test_gen)
# model.fit(train_gen, test_gen, epochs)

model.save_weights('init_model.h5')

Epoch 1/50
84/84 [==============================] - 99s 1s/step - loss: 1.6769 - accuracy: 0.6715 - val_loss: 0.4854 - val_accuracy: 0.7845
Epoch 2/50
84/84 [==============================] - 81s 966ms/step - loss: 0.4946 - accuracy: 0.8125 - val_loss: 0.3625 - val_accuracy: 0.8163
Epoch 3/50
84/84 [==============================] - 79s 939ms/step - loss: 0.4483 - accuracy: 0.7935 - val_loss: 0.3117 - val_accuracy: 0.8657
Epoch 4/50
84/84 [==============================] - 76s 898ms/step - loss: 0.3280 - accuracy: 0.8652 - val_loss: 0.2957 - val_accuracy: 0.8799
Epoch 5/50
84/84 [==============================] - 78s 932ms/step - loss: 0.3448 - accuracy: 0.8425 - val_loss: 0.3716 - val_accuracy: 0.8481
Epoch 6/50
84/84 [==============================] - 79s 941ms/step - loss: 0.3688 - accuracy: 0.8386 - val_loss: 0.3616 - val_accuracy: 0.8163
Epoch 7/50
84/84 [==============================] - 78s 931ms/step - loss: 0.2732 - accuracy: 0.8761 - val_loss: 0.2145 - val_accuracy: 0.9258
Ep

In [ ]:
# import cv2
# img = 'data_set/66.jpg'
# img = cv2.resize(cv2.imread(img), (32,32)) / 255.0
# prediction = model.predict(img.reshape(1,32,32,3))
# print(prediction)
# classes[prediction.argmax()]

In [52]:
image_pred = image.load_img('data_set/26.jpg', target_size = (img_width, img_width))
image_pred = image.image.img_to_array(image_pred)
image_pred = np.expand_dims(image_pred, axis=0)

result = model.predict(image_pred)
# print(result)  #Amelanotic melanoma   Hypomelanotic malignant melanoma.jpg

classes[result.argmax()]

'Nevus'